# 26. Tabularディープラーニング - Neural Networks for Tabular Data

## 概要
テーブルデータに対してニューラルネットワークを適用し、GBDTとの性能比較を行います。

## 学習目標
- Tabularデータでのディープラーニングの特徴を理解できる
- PyTorchでシンプルなニューラルネットワークを実装できる
- GBDTとの比較ができる
- 実務での使い分けができる

In [1]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import lightgbm as lgb
import xgboost as xgb

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

import warnings
warnings.filterwarnings('ignore')

# 設定
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)
torch.manual_seed(42)

# デバイス設定
device = torch.device('cpu')

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

PyTorch version: 2.9.1
Device: cpu


## 1. Tabularデータとディープラーニング

### Tabularデータの特徴

テーブルデータ（行列形式）は最も一般的なデータ形式です。

### 従来の常識

**GBDT（Gradient Boosting Decision Trees）が最強**
- XGBoost、LightGBM、CatBoost
- Kaggleコンペで圧倒的な実績
- 特徴量エンジニアリング不要

### ディープラーニングの可能性

最近の研究により、適切に設計すればNNもGBDTと同等以上の性能を発揮できることが分かってきました。

**いつNNを使うべきか？**
- データサイズが大きい（10万サンプル以上）
- 埋め込み学習が必要
- エンドツーエンドの学習が必要
- GPUが使える環境

## 2. データの準備

In [2]:
# データセットの読み込み（乳がん診断データ）
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

print(f"データサイズ: {X.shape}")
print(f"特徴量数: {X.shape[1]}")
print(f"サンプル数: {X.shape[0]}")
print(f"\nターゲット分布:")
print(y.value_counts())
print(f"\n特徴量の例（最初の5列）:")
print(X.iloc[:5, :5])

データサイズ: (569, 30)
特徴量数: 30
サンプル数: 569

ターゲット分布:
1    357
0    212
Name: count, dtype: int64

特徴量の例（最初の5列）:
   mean radius  mean texture  mean perimeter  mean area  mean smoothness
0        17.99         10.38          122.80     1001.0          0.11840
1        20.57         17.77          132.90     1326.0          0.08474
2        19.69         21.25          130.00     1203.0          0.10960
3        11.42         20.38           77.58      386.1          0.14250
4        20.29         14.34          135.10     1297.0          0.10030


In [3]:
# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# さらに検証データを分割
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"訓練データ: {X_train.shape}")
print(f"検証データ: {X_val.shape}")
print(f"テストデータ: {X_test.shape}")

# スケーリング（ディープラーニングには必須）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\n重要: ディープラーニングでは特徴量のスケーリングが必須")
print("訓練データでfitし、検証・テストデータはtransformのみ")

訓練データ: (364, 30)
検証データ: (91, 30)
テストデータ: (114, 30)

重要: ディープラーニングでは特徴量のスケーリングが必須
訓練データでfitし、検証・テストデータはtransformのみ


## 3. シンプルなニューラルネットワークの構築

### アーキテクチャの設計

Tabularデータには深すぎないネットワークが有効です。

```
Input (30 features)
    ↓
Linear(30 → 64) + ReLU + Dropout(0.3)
    ↓
Linear(64 → 32) + ReLU + Dropout(0.2)
    ↓
Linear(32 → 2)
    ↓
Output (2 classes)
```

In [4]:
# シンプルなニューラルネットワーク
class SimpleTabularNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1=64, hidden_dim2=32, output_dim=2, dropout1=0.3, dropout2=0.2):
        super(SimpleTabularNN, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout1)
        self.dropout2 = nn.Dropout(dropout2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        
        x = self.fc3(x)
        return x

# モデルの初期化
input_dim = X_train_scaled.shape[1]
model = SimpleTabularNN(input_dim=input_dim)
model = model.to(device)

print("モデルアーキテクチャ:")
print(model)
print(f"\nパラメータ数: {sum(p.numel() for p in model.parameters()):,}")

モデルアーキテクチャ:
SimpleTabularNN(
  (fc1): Linear(in_features=30, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
  (dropout1): Dropout(p=0.3, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
)

パラメータ数: 4,130


## 4. 学習の準備

In [5]:
# データローダーの準備
def create_dataloader(X, y, batch_size=32, shuffle=True):
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.LongTensor(y.values if isinstance(y, pd.Series) else y)
    dataset = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_loader = create_dataloader(X_train_scaled, y_train, batch_size=32, shuffle=True)
val_loader = create_dataloader(X_val_scaled, y_val, batch_size=32, shuffle=False)
test_loader = create_dataloader(X_test_scaled, y_test, batch_size=32, shuffle=False)

# 損失関数とオプティマイザ
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

print("学習の準備完了")
print(f"- 損失関数: CrossEntropyLoss")
print(f"- オプティマイザ: Adam (lr=0.001)")
print(f"- バッチサイズ: 32")

学習の準備完了
- 損失関数: CrossEntropyLoss
- オプティマイザ: Adam (lr=0.001)
- バッチサイズ: 32


## 5. 学習の実行

In [ ]:
# 学習関数
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += y_batch.size(0)
        correct += predicted.eq(y_batch).sum().item()
    
    return total_loss / len(loader), correct / total

# 評価関数
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y_batch.size(0)
            correct += predicted.eq(y_batch).sum().item()
            
            probs = torch.softmax(outputs, dim=1)[:, 1]
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    
    auc = roc_auc_score(all_labels, all_probs)
    return total_loss / len(loader), correct / total, auc

print("学習関数の準備完了")

学習関数の準備完了


: 

In [ ]:
# 学習の実行
num_epochs = 100
best_val_auc = 0
patience = 15
patience_counter = 0

history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'val_auc': []
}

print("学習開始...\n")

for epoch in range(num_epochs):
    # 訓練
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 検証
    val_loss, val_acc, val_auc = evaluate(model, val_loader, criterion, device)
    
    # スケジューラの更新
    scheduler.step(val_loss)
    
    # 履歴の保存
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_auc'].append(val_auc)
    
    # Early stopping
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        best_epoch = epoch
        patience_counter = 0
        # ベストモデルの保存
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}")
        print(f"  Best Val AUC: {best_val_auc:.4f} (Epoch {best_epoch+1})")
    
    if patience_counter >= patience:
        print(f"\nEarly stopping at epoch {epoch+1}")
        break

# ベストモデルをロード
model.load_state_dict(best_model_state)

print(f"\n学習完了")
print(f"最良エポック: {best_epoch+1}")
print(f"最良検証AUC: {best_val_auc:.4f}")

## 6. 学習曲線の可視化

In [ ]:
# 学習曲線の可視化
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss曲線
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].axvline(best_epoch, color='red', linestyle='--', 
                label=f'Best Epoch ({best_epoch+1})', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss', fontsize=11)
axes[0].set_title('Training and Validation Loss', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy曲線
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_acc'], label='Validation Accuracy', linewidth=2)
axes[1].axvline(best_epoch, color='red', linestyle='--', 
                label=f'Best Epoch ({best_epoch+1})', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('Accuracy', fontsize=11)
axes[1].set_title('Training and Validation Accuracy', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

# AUC曲線
axes[2].plot(history['val_auc'], label='Validation AUC', linewidth=2, color='orange')
axes[2].axvline(best_epoch, color='red', linestyle='--', 
                label=f'Best Epoch ({best_epoch+1})', linewidth=2)
axes[2].set_xlabel('Epoch', fontsize=11)
axes[2].set_ylabel('AUC', fontsize=11)
axes[2].set_title('Validation AUC', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n観察:")
print(f"- 最良エポック: {best_epoch+1}")
print(f"- Early stoppingにより過学習を防止")
print(f"- 検証AUCは {best_val_auc:.4f} に到達")

## 7. テストデータでの評価

In [ ]:
# テストデータで評価
model.eval()
y_pred_nn = []
y_proba_nn = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        _, predicted = outputs.max(1)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        
        y_pred_nn.extend(predicted.cpu().numpy())
        y_proba_nn.extend(probs.cpu().numpy())

y_pred_nn = np.array(y_pred_nn)
y_proba_nn = np.array(y_proba_nn)

# 評価
accuracy_nn = accuracy_score(y_test, y_pred_nn)
auc_nn = roc_auc_score(y_test, y_proba_nn)

print("=" * 60)
print("Neural Networkの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_nn:.4f}")
print(f"ROC-AUC:  {auc_nn:.4f}")
print("\n混同行列:")
print(confusion_matrix(y_test, y_pred_nn))
print("\n詳細レポート:")
print(classification_report(y_test, y_pred_nn, 
                           target_names=['Malignant', 'Benign']))

## 8. GBDTとの比較

In [ ]:
# LightGBM
print("LightGBMの学習中...")
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=5,
    random_state=42,
    verbose=-1
)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(20), lgb.log_evaluation(0)]
)

y_pred_lgb = lgb_model.predict(X_test)
y_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]

accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
auc_lgb = roc_auc_score(y_test, y_proba_lgb)

print("=" * 60)
print("LightGBMの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_lgb:.4f}")
print(f"ROC-AUC:  {auc_lgb:.4f}")

# XGBoost
print("\nXGBoostの学習中...")
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=5,
    random_state=42,
    eval_metric='logloss'
)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

y_pred_xgb = xgb_model.predict(X_test)
y_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
auc_xgb = roc_auc_score(y_test, y_proba_xgb)

print("\n" + "=" * 60)
print("XGBoostの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"ROC-AUC:  {auc_xgb:.4f}")

In [ ]:
# 性能比較
comparison_df = pd.DataFrame([
    {'Model': 'Neural Network', 'Accuracy': accuracy_nn, 'ROC-AUC': auc_nn},
    {'Model': 'LightGBM', 'Accuracy': accuracy_lgb, 'ROC-AUC': auc_lgb},
    {'Model': 'XGBoost', 'Accuracy': accuracy_xgb, 'ROC-AUC': auc_xgb}
]).sort_values('ROC-AUC', ascending=False)

print("\n" + "=" * 60)
print("モデル性能比較")
print("=" * 60)
print(comparison_df.to_string(index=False))

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
colors = ['steelblue', 'lightgreen', 'coral']
axes[0].bar(comparison_df['Model'], comparison_df['Accuracy'], 
            alpha=0.7, edgecolor='black', color=colors)
axes[0].set_ylabel('Accuracy', fontsize=11)
axes[0].set_title('Accuracy Comparison', fontsize=12, fontweight='bold')
axes[0].set_ylim([0.9, 1.0])
axes[0].grid(axis='y', alpha=0.3)
for i, row in comparison_df.iterrows():
    axes[0].text(row.name, row['Accuracy'], f"{row['Accuracy']:.4f}",
                ha='center', va='bottom', fontweight='bold')

# ROC-AUC
axes[1].bar(comparison_df['Model'], comparison_df['ROC-AUC'], 
            alpha=0.7, edgecolor='black', color=colors)
axes[1].set_ylabel('ROC-AUC', fontsize=11)
axes[1].set_title('ROC-AUC Comparison', fontsize=12, fontweight='bold')
axes[1].set_ylim([0.9, 1.0])
axes[1].grid(axis='y', alpha=0.3)
for i, row in comparison_df.iterrows():
    axes[1].text(row.name, row['ROC-AUC'], f"{row['ROC-AUC']:.4f}",
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n観察:")
print("- ニューラルネットワークはGBDTと競争力のある性能を示す")
print("- 小規模データではGBDTも依然として強力")
print("- データサイズや問題によって最適なモデルは異なる")

## 9. 特徴量重要度の比較

In [ ]:
# LightGBMの特徴量重要度
lgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

# XGBoostの特徴量重要度
xgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# LightGBM
top_lgb = lgb_importance.head(10)
axes[0].barh(range(len(top_lgb)), top_lgb['importance'], color='lightgreen', edgecolor='black')
axes[0].set_yticks(range(len(top_lgb)))
axes[0].set_yticklabels(top_lgb['feature'], fontsize=9)
axes[0].set_xlabel('Importance', fontsize=11)
axes[0].set_title('LightGBM - Top 10 Features', fontsize=12, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(axis='x', alpha=0.3)

# XGBoost
top_xgb = xgb_importance.head(10)
axes[1].barh(range(len(top_xgb)), top_xgb['importance'], color='coral', edgecolor='black')
axes[1].set_yticks(range(len(top_xgb)))
axes[1].set_yticklabels(top_xgb['feature'], fontsize=9)
axes[1].set_xlabel('Importance', fontsize=11)
axes[1].set_title('XGBoost - Top 10 Features', fontsize=12, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTop 5 重要な特徴量:")
print("\nLightGBM:")
for i, row in lgb_importance.head(5).iterrows():
    print(f"  {row['feature']:30s}: {row['importance']:.4f}")

print("\nXGBoost:")
for i, row in xgb_importance.head(5).iterrows():
    print(f"  {row['feature']:30s}: {row['importance']:.4f}")

## 10. まとめ

### 本ノートブックで学んだこと

1. **Tabularデータでのディープラーニング**
   - シンプルなアーキテクチャで十分な性能
   - スケーリングが必須
   - Dropout、Early stoppingで過学習防止

2. **GBDTとの比較**
   - NNもGBDTと競争力のある性能
   - データサイズや問題によって最適なモデルは異なる
   - 両方試して比較するのがベスト

3. **実務での使い分け**
   - 小規模データ（<10k）: GBDT推奨
   - 大規模データ（>100k）: NN検討
   - GPU利用可能: NN有利
   - 解釈可能性重視: GBDTの特徴量重要度

### 重要なポイント

- スケーリング必須（StandardScaler推奨）
- Early stoppingで過学習を防ぐ
- 深すぎないネットワーク（2-3層）
- まずGBDTを試し、NNで改善を図る

### 次のステップ

- より複雑なアーキテクチャの実験
- ハイパーパラメータチューニング
- Kaggleコンペでの実践
- TabNet、FT-Transformerなど専用アーキテクチャの学習